In [99]:
import pandas as pd
import time
import string
from functions.capitalization import titlecase
so_file='BIGBANG1.txt'
widths = [4,2,15,10,30,100,4,30,3,7,1,30,30,2,20,2,2,10,9,50,3,30,50]
update=pd.read_fwf(so_file,
                   header=None, 
                   widths=widths,
                   names=['dir_code','so_type','sonumber','mem_wstd','first_name','last_name','sam_city'
                          ,'sam_stname','sam_stsubt','sam_stnmfr','sam_subd','sam_estate','sam_bldname','sam_aptcod','sam_aprmnt'
                          ,'list_code','acc_type','old_wstd','so_date','joint_user','regulatory_id','account_no','distribution_code'],
                   converters = {'mem_wstd': str, 'sam_stnmfr':str,'account_no':str,'old_wstd':str,'so_date':str},
                   index_col=None
                  )
#update.()
update = update.fillna('')
update['acc_type']=update['acc_type'].astype('category')

classes=pd.read_excel('Company_Class.xlsx')
classes.rename(columns={'name1': 'last_name', 'name2': 'first_name'}, inplace=True)
classes_up=classes.copy()
classes_up.last_name=classes_up.last_name.str.upper()
classes_up.first_name=classes_up.first_name.str.upper()
classes_up.fillna(value='', inplace=True)
classes_up.Phone=classes_up.Phone.astype('int64')
#classes_up.class_code=classes_up.class_code.astype('int64')
classes_up = classes_up.drop(['Areacode'], 1)
database_up=update.copy()
database_up.last_name=database_up.last_name.str.upper()
database_up.first_name=database_up.first_name.str.upper()
#database_up.update(classes_up)
database_up['Phone']=database_up.mem_wstd.str.slice(-7).astype('int64')
database_coded=pd.merge(database_up,classes_up, on=['Phone','last_name','first_name'], how='left')
database_coded.last_name=update.last_name
database_coded = database_coded.drop(['Phone'], 1)
#database_coded.fillna(value='', inplace=True)
#database_coded.class_code=database_coded.class_code.astype('str')
#database_coded.fillna(value='', inplace=True)
database_coded.class_code=database_coded.class_code.astype('object')
update=database_coded
update.sort_values(by='last_name',inplace=True)
update.reset_index(drop=True,inplace=True)
#update['acc_type']=update['acc_type'].astype('category')
#update['class_code']=''
update['src']=so_file.split('/')[-1]
update['so_rangedate']=time.strftime("%Y/%m/%d")
rr=update

In [ ]:
#rr.loc[rr.dir_code == 'MM','so_type']=400
rr_crm

In [36]:
rr_crm=pd.DataFrame()
rr_crm['Areacode']=rr.mem_wstd.str.slice(0,-7)#.astype('int64')
rr_crm['Phone']=rr.mem_wstd.str.slice(-7)
rr_crm['name1']=rr.last_name
rr_crm['name2']=rr.first_name
rr_crm['SAM_BLDNAME']=rr.sam_bldname
rr_crm['SAM_STNMFR']=rr.sam_stnmfr
rr_crm['SAM_STNAME']=rr.sam_stname
rr_crm['SAM_STSUBT']=rr.sam_stsubt
rr_crm['sam_estate']=rr.sam_estate
rr_crm['City']=rr.distribution_code.str.split('  ',1).str.get(1).apply(lambda x : x.strip()).str.upper()
rr_crm['Province']=rr.distribution_code.str.split('  ',1).str.get(0).apply(lambda x : x.strip()).str.upper()
rr_crm['class_code']=rr.class_code
rr_crm['acc_type']=rr.acc_type
#rr_crm['Areacode']=rr_crm['Areacode'].astype('int64')
product=pd.read_excel('Product_Lookup.xlsx')
product=product[product.acc_type == 'BR']
product=product.drop('acc_type',1)
product.rename(columns={'PROVINCE': 'Province', 'AREACODE': 'Areacode','PRODUCT SECTION':'Product'}, inplace=True)
product.Province=product.Province.str.upper()
rr_crm.Areacode=rr_crm.Areacode.astype('int64')
rr_crm = pd.merge(rr_crm,product, on=['Province','Areacode'], how='inner')
rr_crm.Areacode=rr_crm.Areacode.astype('object')
rr_crm.name1 = rr_crm.name1.apply(lambda x: titlecase(x) if x.isupper() else x)
rr_crm.name2 = rr_crm.name2.apply(lambda x: titlecase(x) if x.isupper() else x)
rr_crm.SAM_BLDNAME = rr_crm.SAM_BLDNAME.apply(lambda x: titlecase(x) if x.isupper() else x)
rr_crm.SAM_STNAME = rr_crm.SAM_STNAME.apply(lambda x: titlecase(x))
rr_crm.SAM_STSUBT = rr_crm.SAM_STSUBT.apply(lambda x: titlecase(x))
rr_crm.sam_estate = rr_crm.sam_estate.apply(lambda x: titlecase(x))
rr_crm.City = rr_crm.City.apply(lambda x: titlecase(x))
rr_crm.Province = rr_crm.Province.apply(lambda x: titlecase(x))

In [40]:
rr_crm= rr_crm.replace('', np.nan)
rr_crm.name2[rr_crm.name2.notnull()]

Series([], Name: name2, dtype: float64)

In [24]:
import numpy as np
def or_call(crm):
    index=crm.duplicated(['name1','name2','City','Province','SAM_BLDNAME','SAM_STNAME'])
    skip=False
    for i in xrange(len(index)):
        if index.ix[i] == True:
            line=crm.ix[i]
            prev_line=crm.ix[i-1]
            try:
                next_line=crm.ix[i+1]
            except:
                pass
            line['SAM_BLDNAME']=''
            line['SAM_STNMFR']=''
            line['SAM_STNAME']=''
            line['SAM_STSUBT']=''
            line['sam_estate']=''
            line['PROVINCE']=''
            line['class_code']=''
            line['class_desc']=''
            #line['name1']=''
            #line['name2']=''
            if skip == False:
                prev_line['Phone']=str(prev_line.Phone)+'/'+str(line.Phone)
                line['Phone']=''

                if index.ix[i-1] == False and index.ix[i+1]==True:
                    next_line['City']='Or Call:'
                try:
                    if index.ix[i+1] == True:
                        skip =True
                    
                except:
                    pass
            else:
                skip = False
    crm['Phone'].replace('', np.nan, inplace=True)
    return  crm[crm.Phone.notnull()]

    

In [ ]:
import csv 
with open('areacodes') as csvfile:
    reader= csv.reader(csvfile)
    codes = dict((rows[0],rows[1]) for rows in reader)


In [ ]:
codes.get('bulacan')

In [ ]:
print rr_crm.ix[5]
print rr_crm.ix[6]

In [ ]:
rr_crm.duplicated('name1')

In [ ]:
def or_call(crm):
    index=crm.duplicated(['name1','name2','City','Province','SAM_BLDNAME','SAM_STNAME'])
    skip=False
    for i in xrange(len(index)):
        if index.iloc[i] == True:
            line=crm.ix[i]
            prev_line=crm.ix[i-1]
            try:
                next_line=crm.ix[i+1]
            except:
                pass
            line['SAM_BLDNAME']=''
            line['SAM_STNMFR']=''
            line['SAM_STNAME']=''
            line['SAM_STSUBT']=''
            line['sam_estate']=''
            line['PROVINCE']=''
            line['class_code']=''
            line['class_desc']=''
            #line['name1']=''
            #line['name2']=''
            if skip == False:
                prev_line['Phone']=str(prev_line.Phone)+'/'+str(line.Phone)
                line['Phone']=np.nan

                if index.ix[i-1] == False and index.ix[i+1]==True:
                    next_line['City']='Or Call:'
                try:
                    if index.ix[i+1] == True:
                        skip =True
                except:
                    pass
            else:
                skip = False
    #crm['Phone'].replace('', np.nan, inplace=True)
    return  crm[crm.Phone.notnull()]
or_call(rr_crm)

/Users/ncolina/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/ncolina/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/ncolina/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/ncolina/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:15: SettingWithCopyWarning: 
A value is trying to 

In [ ]:
str(open('small_words').read())

In [ ]:
open('small_words').read().rstrip('\n')


In [ ]:
import functions.interface_functions

In [ ]:
import glob
import os
newest = min(glob.iglob('database_backups/*.hdf5'), key=os.path.getctime)


In [ ]:
newest

In [ ]:
rr.distribution_code.str.split('    ',1).str.get(1)

In [ ]:
index=rr_crm.duplicated(['name1','name2','City','Province','SAM_BLDNAME','SAM_STNAME'])


In [ ]:
range(len(index))

In [ ]:
index[1]

In [ ]:
index=rr_crm.duplicated(['name1','name2','City','Province','SAM_BLDNAME','SAM_STNAME'])


In [ ]:
'dfdsf sdfsdfs        sdfsd'.split('s')

In [ ]:
rr

In [ ]:
x=8
x=x or 5
x

In [ ]:
os.getlogin()

In [ ]:
import csv
reader = csv.reader(open('config/input_format'))#,delimiter=',',quotechar='')
titles=reader.next()
widths=reader.next()

In [ ]:
titles[0]

In [ ]:
reader= csv.reader(open('config/input_format'),quoting=csv.QUOTE_NONNUMERIC)
lines=[]
for row in reader:
    print list(row)
names=lines[0]
widths=lines[1]

In [44]:
import ConfigParser
Config = ConfigParser.ConfigParser()
Config.read("config/input_format")
Config.read('config/areacodes')
def ConfigSectionMap(section):
    dict1 = {}
    options = Config.options(section)
    for option in options:
        try:
            dict1[option] = Config.get(section, option)
            if dict1[option] == -1:
                DebugPrint("skip: %s" % option)
        except:
            print("exception on %s!" % option)
            dict1[option] = None
    return dict1

In [45]:
Config.options('areacodes')
#ConfigSectionMap('input_format').keys()

['bulacan',
 'pampanga',
 'benguet',
 'mountain province',
 'albay',
 'camarines norte',
 'camarines sur',
 'sorsogon',
 'agusan del norte',
 'agusan del sur',
 'davao del norte',
 'davao del sur',
 'misamis oriental',
 'sarsagani',
 'south cotabato',
 'misamis occidental',
 'surigao del norte',
 'zamboanga del norte',
 'zamboanga del sur',
 'iloilo',
 'leyte',
 'samar',
 'cebu',
 'maguindanao',
 'sultan kudarat',
 'aklan',
 'bohol',
 'laguna',
 'negros oriental',
 'quezon',
 'surigao del sur',
 'catanduanes',
 'bataan',
 'nueva ecija',
 'tarlac',
 'zambales',
 'abra',
 'cagayan',
 'ifugao',
 'ilocos norte',
 'ilocos sur',
 'isabela',
 'kalinga',
 'la union',
 'nueva vizcaya',
 'pangasinan (central)',
 'pangasinan (eastern)',
 'pangasinan (western)',
 'quirino',
 'batangas',
 'cavite',
 'marinduque',
 'mindoro occidental',
 'mindoro oriental',
 'lanao del norte',
 'north cotabato',
 'negros occidental',
 'antique',
 'capiz',
 'sarangani',
 'pangasinan(central)',
 'pangasinan(eastern)',

In [ ]:
ConfigSectionMap('areacodes')
    

In [56]:
import pandas as pd
import numpy as np
crm=pd.DataFrame(np.array([['AREACODE',10],['b',1],['c',None],['n',5]]))

In [57]:
crm.sort_values(by=0)
crm.sort_values(by=0).drop_duplicates(0,keep='last')

,0,1
0,AREACODE,10
1,b,1
2,c,None
3,n,5


In [369]:
OP= rr[rr.acc_type=='BR']
rr.loc[rr.account_no.isin(OP.account_no),'dir_code']=1
rr

,dir_code,so_type,sonumber,mem_wstd,first_name,last_name,sam_city,sam_stname,sam_stsubt,sam_stnmfr,...,list_code,acc_type,old_wstd,so_date,joint_user,regulatory_id,account_no,distribution_code,src,so_rangedate
0,1,,,27063050,,,MAND,EDSA,AVE,,...,PB,BR,,,,PLD,653096224,METRO MANILA MANDALUYONG CITY,BIGBANG1.txt,2016/07/27
1,1,,,23983090,,"""K"" LINE LOGISTICS (PHILS.), INC.",PARQ,NINOY AQUINO,AVE,,...,PB,BR,,,,PLD,652611419,METRO MANILA PARANAQUE CITY,BIGBANG1.txt,2016/07/27
2,1,,,28251854,,"""K"" LINE LOGISTICS (PHILS.), INC.",PARQ,NINOY AQUINO,AVE,,...,PB,BR,,,,PLD,6782191,METRO MANILA PARANAQUE CITY,BIGBANG1.txt,2016/07/27
3,1,,,25562917,,"""K"" Line Maritime Training Corporation",PASY,CORALWAY,DRV,,...,PB,BR,,,,PLD,195847231,METRO MANILA PASAY CITY,BIGBANG1.txt,2016/07/27
4,1,,,26318181,,(BANK OF THE PHIL ISLANDS),MAND,UNITED,ST,,...,PB,BR,,,,PLD,6274366,METRO MANILA MANDALUYONG CITY,BIGBANG1.txt,2016/07/27
5,1,10,10,10,10,10,10,10,10,10,...,10,GO,,,,PLD,930393,METRO MANILA MAKATI CITY,BIGBANG1.txt,2016/07/27
6,1,10,10,10,10,10,10,10,10,10,...,10,GO,,,,PLD,85034774,METRO MANILA MAKATI CITY,BIGBANG1.txt,2016/07/27
7,1,10,10,10,10,10,10,10,10,10,...,10,GO,,,,PLD,30236157,METRO MANILA MAKATI CITY,BIGBANG1.txt,2016/07/27
8,1,10,10,10,10,10,10,10,10,10,...,10,GO,,,,PLD,123285,METRO MANILA MAKATI CITY,BIGBANG1.txt,2016/07/27
9,1,10,10,10,10,10,10,10,10,10,...,10,GO,,,,PLD,81940932,METRO MANILA MAKATI CITY,BIGBANG1.txt,2016/07/27


In [245]:
product=pd.read_excel('Company_Class.xlsx')
product.name1=product.name1.str.upper()
#product.Phone=5361557
rr_crm.name1=rr_crm.name1.str.upper()
rr_crm=rr_crm.drop_duplicates('Phone')

In [234]:
rr_crm.drop_duplicates('Phone').Phone=3

/Users/ncolina/anaconda/lib/python2.7/site-packages/pandas/core/generic.py:2698: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [249]:
pd.merge(rr_crm,product, on=['Phone'], how='left')

,AREACODE,Phone,name1_x,name2_x,SAM_BLDNAME,SAM_STNMFR,SAM_STNAME,SAM_STSUBT,sam_estate,City,PROVINCE,class_code_x,class_desc,acc_type,Areacode,name1_y,name2_y,Product,class_code_y
0,NaN,10,NaN,,SM MEGAMALL A,,EDSA,AVE,,MANDALUYONG CITY,ncr,,,BR,2,S C TAN EXPORT CORP,NaN,MLABUS2016_MNLYP,104376


In [240]:
rr_crm['Phone']

0        10
1        10
2        10
3        10
4        10
5        10
6        10
7        10
8        10
9        10
10       10
11       10
12       10
13       10
14       10
15       10
16       10
17       10
18       10
19       10
20       10
21       10
22       10
23       10
24       10
25       10
26       10
27       10
28       10
29       10
         ..
57770    10
57771    10
57772    10
57773    10
57774    10
57775    10
57776    10
57777    10
57778    10
57779    10
57780    10
57781    10
57782    10
57783    10
57784    10
57785    10
57786    10
57787    10
57788    10
57789    10
57790    10
57791    10
57792    10
57793    10
57794    10
57795    10
57796    10
57797    10
57798    10
57799    10
Name: Phone, dtype: int64

In [248]:
product.Phone=10
product=product.drop_duplicates('Phone')

In [263]:
Config.read("config/crm_format")

Config.options('crm_format')

ParsingError: File contains parsing errors: config/crm_format
	[line  2]: 'Areacode\n'
	[line  3]: 'Phone\n'
	[line  4]: 'name1\n'
	[line  5]: 'name2\n'
	[line  6]: 'SAM_BLDNAME\n'
	[line  7]: 'SAM_STNMFR\n'
	[line  8]: 'SAM_STNAME\n'
	[line  9]: 'SAM_STSUBT\n'
	[line 10]: 'sam_estate\n'
	[line 11]: 'City\n'
	[line 12]: 'Province\n'
	[line 13]: 'class_code\n'
	[line 14]: 'acc_type\n'

In [373]:
classes=pd.read_excel('Company_Class.xlsx')
classes_up=classes
classes_up.name1=classes_up.name1.str.upper()
        #remove columns not used to find classes here
classes_up = classes_up.drop(['name2','Product'], 1)
classes_up
#database_up=database.name1.str.upper(inplace=True)
#database_coded=pd.merge(database_up,classes_up, on=['Areacode','Phone','name1'], how='left')
#database_coded.name1=database.name1

,Areacode,Phone,name1,class_code
0,2,7275767,S C TAN EXPORT CORP,104376
1,2,5232306,ARMAK ABRASIVE PRODUCTS,104377
2,2,5311805,CORD CHEMICAL PHILIPPINES INCORPORATED,104377
3,2,6384880,GE-MA MARKETING,104377
4,2,5213883,ROYAL GLOBE CORPORATION,104377
5,2,8102991,SHIMEX TRADING INCORPORATED,104377
6,2,4214883,TRANSPO SALES INC,104377
7,2,9958937,TRANSPO SALES INTERNATIONAL,104377
8,2,2942655,COYNE MERCHANDISING,104384
9,2,8211789,HANDOG MARKETING,104384


In [396]:
classes.Phone.iloc[5]=None

In [401]:
len(classes.index)

69

In [404]:
classes.append()

TypeError: append() takes at least 2 arguments (1 given)

In [407]:
classes

,Areacode,Phone,name1,name2,Product,class_code
0,2,7275767,S C TAN EXPORT CORP,NaN,MLABUS2016_MNLYP,104376
1,2,5232306,ARMAK ABRASIVE PRODUCTS,NaN,MLABUS2016_MNLYP,104377
2,2,5311805,CORD CHEMICAL PHILIPPINES INCORPORATED,NaN,MLABUS2016_MNLYP,104377
3,2,,GE-MA MARKETING,NaN,MLABUS2016_MNLYP,104377
4,2,5213883,ROYAL GLOBE CORPORATION,NaN,MLABUS2016_MNLYP,104377
5,2,None,SHIMEX TRADING INCORPORATED,NaN,MLABUS2016_MNLYP,104377
6,2,4214883,TRANSPO SALES INC,NaN,MLABUS2016_MNLYP,104377
7,2,9958937,TRANSPO SALES INTERNATIONAL,NaN,MLABUS2016_MNLYP,104377
8,2,2942655,COYNE MERCHANDISING,NaN,MLABUS2016_MNLYP,104384
9,2,8211789,HANDOG MARKETING,NaN,MLABUS2016_MNLYP,104384


In [5]:
rr_crm=rr_crm[rr_crm.acc_type=='BR']
acc_type="BR"
product=pd.read_excel('Product_Lookup.xlsx')
product=product[product.acc_type == acc_type]
product=product.drop('acc_type',1)
product.rename(columns={'PROVINCE': 'Province', 'AREACODE': 'Areacode','PRODUCT SECTION':'Product'}, inplace=True)
#product.Province=product.Province.apply(lambda x : titlecase(x)
pd.merge(rr_crm,product, on=['Province','Areacode'], how='inner')


,Areacode,Phone,name1,name2,SAM_BLDNAME,SAM_STNMFR,SAM_STNAME,SAM_STSUBT,sam_estate,City,Province,class_code,class_desc,acc_type,Product


In [452]:
product

,Areacode,Province,Product
2,62,ZAMBOANGA_CITY,ZAMBO2016_ZAMBOCTYWP
5,65,ZAMBOANGA DEL NORTE,ZAMBO2016_ZAMDNWP
8,62,ZAMBOANGA DEL SUR,ZAMBO2016_ZAMDSWP
11,62,ZAMBOANGA SIBUGAY,ZAMBO2016_ZAMSIBWP
14,88,MISAMIS ORIENTAL,MSOCOR2016_MISORWP
17,88,MISAMIS OCCIDENTAL,MSOCOR2016_MISOCWP
20,63,MISAMIS ORIENTAL,PMARTL2016_PMMISORWP
23,63,LANAO DEL NORTE,PMARTL2016_LANAOWP
26,38,BOHOL,BOHLEY2016_BOHOLWP
29,53,LEYTE,BOHLEY2016_LEYTEWP


In [3]:
rr_crm


,Areacode,Phone,name1,name2,SAM_BLDNAME,SAM_STNMFR,SAM_STNAME,SAM_STSUBT,sam_estate,City,Province,class_code,class_desc,acc_type
0,2,7063050,,,Sm Megamall A,,Edsa,Ave,,Mandaluyong City,<bound method StringMethods.<lambda> of <panda...,,,BR
1,2,3983090,"""K"" Line Logistics (Phils.), Inc.",,Oyster Complex Building,,Ninoy Aquino,Ave,,Paranaque City,<bound method StringMethods.<lambda> of <panda...,,,BR
2,2,8251854,"""K"" Line Logistics (Phils.), Inc.",,,,Ninoy Aquino,Ave,,Paranaque City,<bound method StringMethods.<lambda> of <panda...,,,BR
3,2,5562917,"""K"" Line Maritime Training Corporation",,K-Line Building,,Coralway,Drv,CENTRAL BUSINESS PARK 1,Pasay City,<bound method StringMethods.<lambda> of <panda...,,,BR
4,2,6318181,(Bank of the Phil Islands),,Mayflower Plaza Building,,United,St,,Mandaluyong City,<bound method StringMethods.<lambda> of <panda...,,,BR
5,2,8967916,(Department of Trade and Industrybsmbd),,Dti Building,361,Sen Gil Puyat,Ave,,Makati City,<bound method StringMethods.<lambda> of <panda...,,,GO
6,2,8975554,(Department of Trade and Industrycic),,Dti Building,361,Sen Gil Puyat,Ave,Bel-Air Village,Makati City,<bound method StringMethods.<lambda> of <panda...,,,GO
7,2,8908907,(Department of Trade and Industryipr),,Dti Building,361,Sen Gil Puyat,Ave,Bel-Air Village,Makati City,<bound method StringMethods.<lambda> of <panda...,,,GO
8,2,8904951,(Department of Trade and Industryoop),,Industry & Investment Building,385,Sen Gil Puyat,Ave,Bel-Air Village,Makati City,<bound method StringMethods.<lambda> of <panda...,,,GO
9,2,8971719,(Department of Trade and Industryticci),,Lc Building,375,Sen Gil Puyat,Ave,Bel-Air Village,Makati City,<bound method StringMethods.<lambda> of <panda...,,,GO


In [465]:
product=pd.read_excel('Product_Lookup.xlsx')
sum(product.AREACODE==2)


15

In [34]:
import numpy as np
classes=pd.read_excel('Company_Class.xlsx')
classes.rename(columns={'name1': 'last_name', 'name2': 'first_name'}, inplace=True)
classes_up=classes.copy()
classes_up.last_name=classes_up.last_name.str.upper()
classes_up.first_name=classes_up.first_name.str.upper()
classes_up.fillna(value='', inplace=True)
classes_up.Phone=classes_up.Phone.astype('int64')
#classes_up.class_code=classes_up.class_code.astype('int64')
classes_up = classes_up.drop(['Areacode','Product'], 1)
database_up=update.copy()
database_up.last_name=database_up.last_name.str.upper()
database_up.first_name=database_up.first_name.str.upper()
#database_up.update(classes_up)
database_up['Phone']=database_up.mem_wstd.str.slice(-7).astype('int64')
database_coded=pd.merge(database_up,classes_up, on=['Phone','last_name','first_name'], how='left')
database_coded.last_name=update.last_name
database_coded = database_coded.drop(['Phone'], 1)
#database_coded.fillna(value='', inplace=True)
#database_coded.class_code=database_coded.class_code.astype('str')
#database_coded.fillna(value='', inplace=True)
database_coded.class_code=database_coded.class_code.astype('object')
database_coded.dtypes
#database_coded.to_hdf('ha.hdf5', 'database', mode='w', format='fixed')
#database_coded

AttributeError: 'DataFrame' object has no attribute 'class_code'

In [36]:
database_coded#[~database_coded.class_code.isnull()]

,dir_code,so_type,sonumber,mem_wstd,first_name,last_name,sam_city,sam_stname,sam_stsubt,sam_stnmfr,...,old_wstd,so_date,joint_user,regulatory_id,account_no,distribution_code,class_code_x,src,so_rangedate,class_code_y
0,MM,,,27063050,,,MAND,EDSA,AVE,,...,,,,PLD,653096224,METRO MANILA MANDALUYONG CITY,NaN,BIGBANG1.txt,2016/08/01,NaN
1,MM,,,23983090,,"""K"" LINE LOGISTICS (PHILS.), INC.",PARQ,NINOY AQUINO,AVE,,...,,,,PLD,652611419,METRO MANILA PARANAQUE CITY,NaN,BIGBANG1.txt,2016/08/01,NaN
2,MM,,,28251854,,"""K"" LINE LOGISTICS (PHILS.), INC.",PARQ,NINOY AQUINO,AVE,,...,,,,PLD,6782191,METRO MANILA PARANAQUE CITY,NaN,BIGBANG1.txt,2016/08/01,NaN
3,MM,,,25562917,,"""K"" Line Maritime Training Corporation",PASY,CORALWAY,DRV,,...,,,,PLD,195847231,METRO MANILA PASAY CITY,NaN,BIGBANG1.txt,2016/08/01,NaN
4,MM,,,26318181,,(BANK OF THE PHIL ISLANDS),MAND,UNITED,ST,,...,,,,PLD,6274366,METRO MANILA MANDALUYONG CITY,NaN,BIGBANG1.txt,2016/08/01,NaN
5,MM,,,28967916,,(DEPARTMENT OF TRADE AND INDUSTRYBSMBD),MAKT,SEN GIL PUYAT,AVE,361,...,,,,PLD,930393,METRO MANILA MAKATI CITY,NaN,BIGBANG1.txt,2016/08/01,NaN
6,MM,,,28975554,,(DEPARTMENT OF TRADE AND INDUSTRYCIC),MAKT,SEN GIL PUYAT,AVE,361,...,,,,PLD,85034774,METRO MANILA MAKATI CITY,NaN,BIGBANG1.txt,2016/08/01,NaN
7,MM,,,28908907,,(DEPARTMENT OF TRADE AND INDUSTRYIPR),MAKT,SEN GIL PUYAT,AVE,361,...,,,,PLD,30236157,METRO MANILA MAKATI CITY,NaN,BIGBANG1.txt,2016/08/01,NaN
8,MM,,,28904951,,(DEPARTMENT OF TRADE AND INDUSTRYOOP),MAKT,SEN GIL PUYAT,AVE,385,...,,,,PLD,123285,METRO MANILA MAKATI CITY,NaN,BIGBANG1.txt,2016/08/01,NaN
9,MM,,,28971719,,(DEPARTMENT OF TRADE AND INDUSTRYTICCI),MAKT,SEN GIL PUYAT,AVE,375,...,,,,PLD,81940932,METRO MANILA MAKATI CITY,NaN,BIGBANG1.txt,2016/08/01,NaN


In [4]:
x=pd.read_hdf('ha.hdf5')

In [7]:
x[~x.class_code.isnull()]

,dir_code,so_type,sonumber,mem_wstd,first_name,last_name,sam_city,sam_stname,sam_stsubt,sam_stnmfr,...,sam_aprmnt,list_code,acc_type,old_wstd,so_date,joint_user,regulatory_id,account_no,distribution_code,class_code
3227,MM,,,22885101,,ASIATIC ALUMINUM PRODUCTS COMPANY,MALA,CELIA,ST,55,...,,PB,BR,,,,PLD,3728439,METRO MANILA MALABON CITY,104491
6300,MM,,,23710329,,ROOSEVELT ALUMINUM PRODUCTS COMPANY INCORPORATED,QUEO,JUDGE JUAN LUNA,ST,183,...,,PB,BR,,,,PLD,35916962,METRO MANILA QUEZON CITY,104491
13987,MM,,,25310891,,NEW TOPBEST LUMBER CONSTRUCTION SUPPLY INC,MAND,EDSA,AVE,282,...,,PB,BR,,,,PLD,6338607,METRO MANILA MANDALUYONG CITY,104491
18002,MM,,,25842303,,"ePLDT, Inc.",PASI,CANLEY,RD,,...,,PB,BR,,,,PLD,651952212,METRO MANILA PASIG CITY,6.9697e+06
23944,MM,,,26712239,,ITX DISTRIBUTORS CORPORATION,PASI,ROSEMARIE,LNE,5,...,,PB,BR,,,,PLD,7271654,METRO MANILA PASIG CITY,104437
32488,MM,,,28102991,,SHIMEX TRADING INCORPORATED,MAKT,CHINO ROCES,AVE,2263,...,305,PB,BR,,,,PLD,6823262,METRO MANILA MAKATI CITY,104377
34986,MM,,,28161186,,AGCHEM MFG CORP,MAKT,A S ARNAIZ,AVE,835,...,,PB,BR,,,,PLD,6702074,METRO MANILA MAKATI CITY,104433
36849,MM,,,28222529,,AAI WORLDWIDE LOGISTICS INC.,PARQ,KAINGIN,RD,,...,,PB,BR,,,,PLD,80398727,METRO MANILA PARANAQUE CITY,100000
41954,MM,,,28448441,,CHAN C BROS INC,PASY,TOLENTINO,ST,2225,...,,PB,BR,,,LIWAYWAY MARKETING CORPORATION,PLD,30813820,METRO MANILA PASAY CITY,104392
53467,MM,,,29217686,,CHARTERED ADJUSTERS INCORPORATED,QUEO,EDSA,AVE,784,...,5,PB,BR,,,,PLD,28800541,METRO MANILA QUEZON CITY,104393


In [10]:
line=x.ix[1]
line.sam_city=''
line

dir_code                                                    MM
so_type                                                       
sonumber                                                      
mem_wstd                                              28322039
first_name                                                    
last_name                         Home Development Mutual Fund
sam_city                                                      
sam_stname                                       SEN GIL PUYAT
sam_stsubt                                                 AVE
sam_stnmfr                                                 358
sam_subd                                                      
sam_estate                                     SALCEDO VILLAGE
sam_bldname                                  PETRON MEGA PLAZA
sam_aptcod                                                    
sam_aprmnt                                                    
list_code                                              

In [29]:
def or_call(crm):
    crm.reset_index(drop=True,inplace=True)
    index=crm.duplicated(['name1','name2','City','Province','SAM_BLDNAME','SAM_STNAME'])
    skip=False
    for i in xrange(len(index)):
        if index.iloc[i] == True:
            line=crm.ix[i]
            prev_line=crm.ix[i-1]
            try:
                next_line=crm.iloc[i+1]
            except:
                pass
            crm.SAM_BLDNAME.iloc[i]=''
            crm.SAM_STNMFR.iloc[i]=''
            crm.SAM_STNAME.iloc[i]=''
            crm.SAM_STSUBT.iloc[i]=''
            crm.sam_estate.iloc[i]=''
            crm.Province.iloc[i]=''
            crm.class_code.iloc[i]=''
            crm.Product.iloc[i]=''

            if skip == False:
                crm.Phone.iloc[i-1]=str(prev_line.Phone)+'/'+str(line.Phone)
                crm.Phone.iloc[i]=np.nan
                if index.iloc[i-1] == False and index.iloc[i+1]==True:
                    crm.City.iloc[i+1]='Or Call:'
                else:
                    crm.City.iloc[i+1]=''

                try:
                    if index.iloc[i+1] == True:
                        skip =True
                except:
                    pass
            else:
                skip = False
    return  crm[crm.Phone.notnull()]


In [31]:
update.dtypes

dir_code             object
so_type              object
sonumber             object
mem_wstd             object
first_name           object
last_name            object
sam_city             object
sam_stname           object
sam_stsubt           object
sam_stnmfr           object
sam_subd             object
sam_estate           object
sam_bldname          object
sam_aptcod           object
sam_aprmnt           object
list_code            object
acc_type             object
old_wstd             object
so_date              object
joint_user           object
regulatory_id        object
account_no           object
distribution_code    object
class_code           object
src                  object
so_rangedate         object
dtype: object

In [43]:
#update=update.head()
#update=update.drop(['so_rangedate','src','class_code'],1)
np.savetxt("testing.txt", update.values, fmt=formats, delimiter='')


AttributeError: fmt has wrong shape.  []

In [47]:
ConfigSectionMap('input_format')

NameError: name 'ConfigSectionMap' is not defined

In [52]:
print '%'+'10' +'s'

%10s


In [46]:
import ConfigParser
Config = ConfigParser.ConfigParser()
Config.read("config/input_format")
formats=[]
for field in Config.options('input_format'):
    formats.append('%-'+str(Config.get('input_format',field)) +'s')

In [41]:
formats

NameError: name 'formats' is not defined

In [98]:
update

RuntimeError: maximum recursion depth exceeded while calling a Python object

RuntimeError: maximum recursion depth exceeded in cmp

In [261]:
from difflib import SequenceMatcher
SequenceMatcher(None,'Food INC','Inc Food').ratio()

0.5

In [30]:
import numpy as np
update=update.sort_values('last_name')
update=update.replace('', np.nan)#, regex=True)
update.last_name.notnull()

7924      True
37404     True
16890     True
19139     True
51037     True
51283     True
47986     True
47853     True
51126     True
47848     True
47860     True
50843     True
51036     True
51114     True
50688     True
50809     True
47831     True
50657     True
47841     True
47748     True
2332      True
7350      True
29308     True
29309     True
29311     True
29312     True
7352      True
7351      True
54336     True
18306     True
         ...  
57706     True
4835      True
43250     True
10722     True
41935     True
10729     True
15048     True
10721     True
42006     True
31851     True
16459     True
47733     True
47732     True
49591     True
9293      True
90        True
93        True
17957     True
87        True
24116     True
18002     True
8317      True
89        True
41905     True
49703     True
49704     True
49702     True
49360     True
49698     True
25216    False
Name: last_name, dtype: bool

In [58]:
update.drop(['src','class_code','so_rangedate'],1).to_records()

rec.array([ (0, 'MM', '', '', '27063050', '', '', 'MAND', 'EDSA', 'AVE', '', '', '', 'SM MEGAMALL A', '', '', 'PB', 'BR', '', '', '', 'PLD', '653096224', 'METRO MANILA                  MANDALUYONG CITY'),
 (1, 'MM', '', '', '23983090', '', '"K" LINE LOGISTICS (PHILS.), INC.', 'PARQ', 'NINOY AQUINO', 'AVE', '', '', '', 'OYSTER COMPLEX BUILDING', '', 'WAREHOUSE 8 BLDG2', 'PB', 'BR', '', '', '', 'PLD', '652611419', 'METRO MANILA                  PARANAQUE CITY'),
 (2, 'MM', '', '', '28251854', '', '"K" LINE LOGISTICS (PHILS.), INC.', 'PARQ', 'NINOY AQUINO', 'AVE', '', '', '', '', '', '', 'PB', 'BR', '', '', '', 'PLD', '6782191', 'METRO MANILA                  PARANAQUE CITY'),
 ...,
 (57797, 'MM', '', '', '28935956', '', 'kgb Philippines, Inc.', 'MAKT', 'AYALA', 'AVE', '6819', '', 'SALCEDO VILLAGE', 'RCBC PLAZA YUCHENGCO TOWER 2', '', '', 'PB', 'BR', '', '', '', 'PLD', '153087512', 'METRO MANILA                  MAKATI CITY'),
 (57798, 'MM', '', '', '28931693', '', 'kgb Philippines, Inc.'

In [253]:
rr_crm[rr_crm.name1.str.contains(""""|#|;|\+|\@|\$|\%|\^|\*|=|\(|\)|\[|\]""")]


,Areacode,Phone,name1,name2,SAM_BLDNAME,SAM_STNMFR,SAM_STNAME,SAM_STSUBT,sam_estate,City,Province,class_code,acc_type,Product
1,2,3983090,"""K"" Line Logistics (Phils.), Inc.",,Oyster Complex Building,,Ninoy Aquino,Ave,,Paranaque City,Metro Manila,,BR,MLABUS2016_MLABUSWP
2,2,8251854,"""K"" Line Logistics (Phils.), Inc.",,,,Ninoy Aquino,Ave,,Paranaque City,Metro Manila,,BR,MLABUS2016_MLABUSWP
3,2,5562917,"""K"" Line Maritime Training Corporation",,K-Line Building,,Coralway,Drv,CENTRAL BUSINESS PARK 1,Pasay City,Metro Manila,,BR,MLABUS2016_MLABUSWP
4,2,6318181,(Bank of the Phil Islands),,Mayflower Plaza Building,,United,St,,Mandaluyong City,Metro Manila,,BR,MLABUS2016_MLABUSWP
5,2,8967916,(Department of Trade and Industrybsmbd),,Dti Building,361,Sen Gil Puyat,Ave,,Makati City,Metro Manila,,GO,MLABUS2016_MLABUSWP
6,2,8975554,(Department of Trade and Industrycic),,Dti Building,361,Sen Gil Puyat,Ave,Bel-Air Village,Makati City,Metro Manila,,GO,MLABUS2016_MLABUSWP
7,2,8908907,(Department of Trade and Industryipr),,Dti Building,361,Sen Gil Puyat,Ave,Bel-Air Village,Makati City,Metro Manila,,GO,MLABUS2016_MLABUSWP
8,2,8904951,(Department of Trade and Industryoop),,Industry & Investment Building,385,Sen Gil Puyat,Ave,Bel-Air Village,Makati City,Metro Manila,,GO,MLABUS2016_MLABUSWP
9,2,8971719,(Department of Trade and Industryticci),,Lc Building,375,Sen Gil Puyat,Ave,Bel-Air Village,Makati City,Metro Manila,,GO,MLABUS2016_MLABUSWP
10,2,8904857,(Dept of Trade & Industryhrds),,Dti Building,,Sen Gil Puyat,Ave,Bel-Air Village,Makati City,Metro Manila,,GO,MLABUS2016_MLABUSWP


In [306]:
update

,dir_code,so_type,sonumber,mem_wstd,first_name,last_name,sam_city,sam_stname,sam_stsubt,sam_stnmfr,...,acc_type,old_wstd,so_date,joint_user,regulatory_id,account_no,distribution_code,class_code,src,so_rangedate
0,MM,,,27063050,,,MAND,EDSA,AVE,,...,BR,,,,PLD,653096224,METRO MANILA MANDALUYONG CITY,NaN,BIGBANG1.txt,2016/08/04
1,MM,,,23983090,,"""K"" LINE LOGISTICS (PHILS.), INC.",PARQ,NINOY AQUINO,AVE,,...,BR,,,,PLD,652611419,METRO MANILA PARANAQUE CITY,NaN,BIGBANG1.txt,2016/08/04
2,MM,,,28251854,,"""K"" LINE LOGISTICS (PHILS.), INC.",PARQ,NINOY AQUINO,AVE,,...,BR,,,,PLD,6782191,METRO MANILA PARANAQUE CITY,NaN,BIGBANG1.txt,2016/08/04
3,MM,,,25562917,,"""K"" Line Maritime Training Corporation",PASY,CORALWAY,DRV,,...,BR,,,,PLD,195847231,METRO MANILA PASAY CITY,NaN,BIGBANG1.txt,2016/08/04
4,MM,,,26318181,,(BANK OF THE PHIL ISLANDS),MAND,UNITED,ST,,...,BR,,,,PLD,6274366,METRO MANILA MANDALUYONG CITY,NaN,BIGBANG1.txt,2016/08/04
5,MM,,,28967916,,(DEPARTMENT OF TRADE AND INDUSTRYBSMBD),MAKT,SEN GIL PUYAT,AVE,361,...,GO,,,,PLD,930393,METRO MANILA MAKATI CITY,NaN,BIGBANG1.txt,2016/08/04
6,MM,,,28975554,,(DEPARTMENT OF TRADE AND INDUSTRYCIC),MAKT,SEN GIL PUYAT,AVE,361,...,GO,,,,PLD,85034774,METRO MANILA MAKATI CITY,NaN,BIGBANG1.txt,2016/08/04
7,MM,,,28908907,,(DEPARTMENT OF TRADE AND INDUSTRYIPR),MAKT,SEN GIL PUYAT,AVE,361,...,GO,,,,PLD,30236157,METRO MANILA MAKATI CITY,NaN,BIGBANG1.txt,2016/08/04
8,MM,,,28904951,,(DEPARTMENT OF TRADE AND INDUSTRYOOP),MAKT,SEN GIL PUYAT,AVE,385,...,GO,,,,PLD,123285,METRO MANILA MAKATI CITY,NaN,BIGBANG1.txt,2016/08/04
9,MM,,,28971719,,(DEPARTMENT OF TRADE AND INDUSTRYTICCI),MAKT,SEN GIL PUYAT,AVE,375,...,GO,,,,PLD,81940932,METRO MANILA MAKATI CITY,NaN,BIGBANG1.txt,2016/08/04


In [272]:
set('x"yz"')#.issubset("xyz")

{'"', 'x', 'y', 'z'}

In [181]:
rr_crm=rr_crm.fillna('')

def capital(str):
    if str.isupper() and ~str.isalpha():
        return titlecase(str)
    else:
        return str

In [275]:
import re

small_word_file =open("config/small_words",'r')
SMALL = str(small_word_file.read().rstrip('\n'))
#ADD USUAL FILIPINO articles
PUNCT = r"""!"#$%&'‘()*+,\-./:;?@[\\\]_`{|}~"""


SMALL_WORDS = re.compile(r'^(%s)$' % SMALL, re.I)
INLINE_PERIOD = re.compile(r'[a-z][.][a-z]', re.I)
UC_ELSEWHERE = re.compile(r'[%s]*?[a-zA-Z]+[A-Z]+?' % PUNCT)
CAPFIRST = re.compile(r"^[%s]*?([A-Za-z])" % PUNCT)
SMALL_FIRST = re.compile(r'^([%s]*)(%s)\b' % (PUNCT, SMALL), re.I)
SMALL_LAST = re.compile(r'\b(%s)[%s]?$' % (SMALL, PUNCT), re.I)
SUBPHRASE = re.compile(r'([:.;?!][ ])(%s)' % SMALL)
APOS_SECOND = re.compile(r"^[dol]{1}['‘]{1}[a-z]+$", re.I)
ALL_CAPS = re.compile(r'^[A-Z\s%s]+$' % PUNCT)
UC_INITIALS = re.compile(r"^(?:[A-Z]{1}\.{1}|[A-Z]{1}\.{1}[A-Z]{1})+$")
MAC_MC = re.compile(r"^([Mm]a?c)(\w+)")

In [280]:
if ALL_CAPS.match('GOOOOOOO'):
    print 'yo'


yo


In [296]:
'WARHOUSE 3B'.isupper()

True

In [324]:
update[update.last_name.str.contains(r'^[-!$%^&*()_+|~=`"{}[\]:/;<>?,.@#]|^$', regex=True)]

,dir_code,so_type,sonumber,mem_wstd,first_name,last_name,sam_city,sam_stname,sam_stsubt,sam_stnmfr,...,acc_type,old_wstd,so_date,joint_user,regulatory_id,account_no,distribution_code,class_code,src,so_rangedate
0,MM,,,27063050,,,MAND,EDSA,AVE,,...,BR,,,,PLD,653096224,METRO MANILA MANDALUYONG CITY,NaN,BIGBANG1.txt,2016/08/04
1,MM,,,23983090,,"""K"" LINE LOGISTICS (PHILS.), INC.",PARQ,NINOY AQUINO,AVE,,...,BR,,,,PLD,652611419,METRO MANILA PARANAQUE CITY,NaN,BIGBANG1.txt,2016/08/04
2,MM,,,28251854,,"""K"" LINE LOGISTICS (PHILS.), INC.",PARQ,NINOY AQUINO,AVE,,...,BR,,,,PLD,6782191,METRO MANILA PARANAQUE CITY,NaN,BIGBANG1.txt,2016/08/04
3,MM,,,25562917,,"""K"" Line Maritime Training Corporation",PASY,CORALWAY,DRV,,...,BR,,,,PLD,195847231,METRO MANILA PASAY CITY,NaN,BIGBANG1.txt,2016/08/04
4,MM,,,26318181,,(BANK OF THE PHIL ISLANDS),MAND,UNITED,ST,,...,BR,,,,PLD,6274366,METRO MANILA MANDALUYONG CITY,NaN,BIGBANG1.txt,2016/08/04
5,MM,,,28967916,,(DEPARTMENT OF TRADE AND INDUSTRYBSMBD),MAKT,SEN GIL PUYAT,AVE,361,...,GO,,,,PLD,930393,METRO MANILA MAKATI CITY,NaN,BIGBANG1.txt,2016/08/04
6,MM,,,28975554,,(DEPARTMENT OF TRADE AND INDUSTRYCIC),MAKT,SEN GIL PUYAT,AVE,361,...,GO,,,,PLD,85034774,METRO MANILA MAKATI CITY,NaN,BIGBANG1.txt,2016/08/04
7,MM,,,28908907,,(DEPARTMENT OF TRADE AND INDUSTRYIPR),MAKT,SEN GIL PUYAT,AVE,361,...,GO,,,,PLD,30236157,METRO MANILA MAKATI CITY,NaN,BIGBANG1.txt,2016/08/04
8,MM,,,28904951,,(DEPARTMENT OF TRADE AND INDUSTRYOOP),MAKT,SEN GIL PUYAT,AVE,385,...,GO,,,,PLD,123285,METRO MANILA MAKATI CITY,NaN,BIGBANG1.txt,2016/08/04
9,MM,,,28971719,,(DEPARTMENT OF TRADE AND INDUSTRYTICCI),MAKT,SEN GIL PUYAT,AVE,375,...,GO,,,,PLD,81940932,METRO MANILA MAKATI CITY,NaN,BIGBANG1.txt,2016/08/04
